### Wine rating classification

In [5]:
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

In [ ]:
mpt = MyPyTable()
filename = "input_data/WineRatings.csv"
mpt.load_from_file(filename)

